In [1]:
import sys, os

sys.path.insert(0, os.path.abspath(os.path.join("..")))

import numpy as np
from uot.problems.generators import GaussianMixtureGenerator

from uot.utils.costs import cost_euclid_squared
from uot.problems.inspect_store import plot_1d

from uot.solvers.gradient_ascent import (
    GradientAscentPlainLogSolver,
    GradientAscentTwoMarginalSolver,
    LogDomainGradientAscentSolver,
)

In [2]:
seed = 55


def get_gaussian_problems(seed):
    gen = GaussianMixtureGenerator(
        name="",
        dim=1,
        num_components=1,
        n_points=64,
        num_datasets=5,
        borders=(-6, 6),
        cost_fn=cost_euclid_squared,
        use_jax=False,
        seed=seed,
    )

    problems = []
    for p in gen.generate():
        problems.append(p)
    return problems

In [3]:
problems_list = get_gaussian_problems(seed=55)

In [ ]:
prob1 = problems_list[0]
mu1, nu1 = prob1.get_marginals()

mu1_pts, mu1_w = mu1.to_discrete()
nu1_pts, nu1_w = nu1.to_discrete()

fig1 = plot_1d(mu1_pts, mu1_w, nu1_pts, nu1_w)
fig1

In [6]:
prob1 = problems_list[0]
gradient_solver = GradientAscentTwoMarginalSolver()
result = gradient_solver.solve(
    marginals=prob1.get_marginals(),
    costs=prob1.get_costs()[0],
    reg=0.1,
    maxiter=10_000,
    tol=1e-6,
    learning_rate=0.001,
)

result

{'transport_plan': Array([[3.0109544e-05, 3.0371319e-05, 3.0948049e-05, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [3.0413927e-05, 3.0678319e-05, 3.1260875e-05, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [3.0760610e-05, 3.1028078e-05, 3.1617215e-05, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        ...,
        [3.0160829e-04, 3.0423049e-04, 3.1000731e-04, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [2.8607761e-04, 2.8856480e-04, 2.9404444e-04, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00],
        [2.7007671e-04, 2.7242478e-04, 2.7759792e-04, ..., 0.0000000e+00,
         0.0000000e+00, 0.0000000e+00]], dtype=float32),
 'cost': Array(0.02570258, dtype=float32),
 'u_final': Array([-4.35586832e-02, -4.25528884e-02, -4.14193645e-02, -4.01458852e-02,
        -3.87198627e-02, -3.71279493e-02, -3.53565551e-02, -3.33915539e-02,
        -3.12191173e-02, -2.88250707e-02, -2.61953883e-02, -2.

# Now with the Experiment Pipeline

In [30]:
from uot.experiments.runner import run_pipeline
from uot.experiments.experiment import Experiment

from uot.solvers.sinkhorn import SinkhornTwoMarginalSolver
from uot.problems.iterator import OnlineProblemIterator
from uot.solvers.solver_config import SolverConfig

In [ ]:
def solve_fn(problem, solver, measures, costs, **kwargs):
    print(problem)
    # print(np.mean(measures[0].to_discrete()[1]))
    # print(np.mean(measures[1].to_discrete()[1]))
    print(measures[0].to_discrete()[1])
    print(measures[1].to_discrete()[1])
    return {}


exp = Experiment(
    name="Testing Generators Reproducibility",
    solve_fn=solve_fn,
)

solvers = [
    SolverConfig(
        name="Sinkhorn",
        solver=SinkhornTwoMarginalSolver,
        param_grid=[
            {
                "reg": 0.01,
            }
        ],
        is_jit=True,
    )
]

seed = 55
iterators = [
    OnlineProblemIterator(
        GaussianMixtureGenerator(
            name="",
            dim=1,
            num_components=1,
            n_points=16,
            num_datasets=2,
            borders=(-6, 6),
            cost_fn=cost_euclid_squared,
            use_jax=False,
            seed=seed,
        ),
        num=2,
        cache_gt=False,
    ),
    OnlineProblemIterator(
        GaussianMixtureGenerator(
            name="",
            dim=1,
            num_components=1,
            n_points=16,
            num_datasets=2,
            borders=(-6, 6),
            cost_fn=cost_euclid_squared,
            use_jax=False,
            seed=seed,
        ),
        num=2,
        cache_gt=False,
    ),
]

run_pipeline(
    experiment=exp,
    solvers=solvers,
    iterators=iterators,
    folds=1,
    progress=True,
)

2025-07-16 18:57:43,918 uot INFO: starting pipeline...


INFO	uot:runner.py:run_pipeline()- starting pipeline...


2025-07-16 18:57:43,929 uot INFO: Running experiments...


INFO	uot:runner.py:run_pipeline()- Running experiments...


2025-07-16 18:57:43,931 uot INFO: Running set of problems on <class 'uot.solvers.sinkhorn.SinkhornTwoMarginalSolver'>            with {'reg': 0.01}


...

<TwoMarginalProblem[] 16x16        with (<map object at 0x130768e50>)>
[2.42353828e-11 8.91080187e-09 1.39215103e-06 9.24184278e-05
 2.60695443e-03 3.12471893e-02 1.59144327e-01 3.44408602e-01
 3.16708326e-01 1.23750694e-01 2.05465276e-02 1.44954573e-03
 4.34538269e-05 5.53511711e-07 2.99590641e-09 6.89020759e-12]
[2.2914280e-06 1.8656305e-05 1.2251979e-04 6.4900506e-04 2.7730081e-03
 9.5568635e-03 2.6566898e-02 5.9569992e-02 1.0773962e-01 1.5717539e-01
 1.8495023e-01 1.7554417e-01 1.3439378e-01 8.2991406e-02 4.1337889e-02
 1.6608279e-02]
2025-07-16 18:57:45,087 uot INFO: Successfully finished SinkhornTwoMarginalSolver with {'reg': 0.01}


INFO	uot:experiment.py:run_on_problems()- Successfully finished SinkhornTwoMarginalSolver with {'reg': 0.01}
Sinkhorn({'reg': 0.01}): 100%|██████████| 4/4 [00:01<00:00,  3.37it/s]

2025-07-16 18:57:45,115 uot INFO: Composing results...



INFO	uot:runner.py:run_pipeline()- Composing results...


,dataset,mu_size,nu_size,cost,status,name,reg
0,,16,16,cost_euclid_squared,success,Sinkhorn,0.01
1,,16,16,cost_euclid_squared,success,Sinkhorn,0.01
2,,16,16,cost_euclid_squared,success,Sinkhorn,0.01
